In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import json
from tensorflow import keras
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_curve, auc

In [2]:
model = keras.models.load_model("../Saved Data/Model")
X_test = np.load('../Saved Data/Test Set/X_test_pre-processed.npy')
y_test = np.load('../Saved Data/Test Set/y_test.npy')

In [3]:
probabilities_vector = model.predict(X_test)
y_pred = np.argmax(probabilities_vector, axis=1)
np.savetxt('../Saved Data/Predicted Labels/python_model.txt', y_pred, fmt='%d')

28/28 [==============================] - 0s 6ms/step


In [4]:
def roc_auc(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])


    return roc_auc

In [5]:
accuracy_score = accuracy_score(y_test, y_pred)
precision_score = precision_score(y_test, y_pred, average='macro')
recall_score = recall_score(y_test, y_pred, average='macro')
f1_score = f1_score(y_test, y_pred, average='macro')
confusion_matrix = confusion_matrix(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)
AUC_score = roc_auc(y_test, y_pred, probabilities_vector, average='macro')['avg / total']

In [6]:
# AUC_scores = roc_auc_score(y_test, probabilities_vector, average=None, multi_class='ovr')
# print(AUC_scores)

# AUC_scores = roc_auc(y_test, y_pred, probabilities_vector, average='macro')
# AUC_scores = list(AUC_scores.values())
# AUC_scores.pop()
# AUC_scores = [round(x, 8) for x in AUC_scores]
# print(AUC_scores)

In [7]:
if os.path.exists('../Saved Data/Temp Data/python_model.json'):
    os.remove('../Saved Data/Temp Data/python_model.json')

In [8]:
data = {"accuracy": accuracy_score, 
        "precision": precision_score, 
        "recall": recall_score, 
        "f1": f1_score, 
        "auc": AUC_score
        }
json_data = json.dumps(data, indent = 4)
json_file = open('../Saved Data/Temp Data/python_model.json', 'w')
json_file.write(json_data)
print(f"Classification Accuracy = {round(accuracy_score * 100, 2)}%\nPrecision = {round(precision_score * 100, 2)}%\nRecall = {round(recall_score * 100, 2)}%\nF1 Score = {round(f1_score, 2)}\nAUC Score = {round(AUC_score, 2)}")
json_file.close()

Classification Accuracy = 87.44%
Precision = 87.17%
Recall = 85.55%
F1 Score = 0.86
AUC Score = 0.98


In [9]:
print(classification_report)

              precision    recall  f1-score   support

           0       0.84      0.91      0.88        93
           1       0.88      0.85      0.86       107
           2       0.93      0.88      0.90        89
           3       0.82      0.83      0.83       139
           4       0.83      0.83      0.83       123
           5       0.89      0.76      0.82        45
           6       0.92      0.93      0.92       280

    accuracy                           0.87       876
   macro avg       0.87      0.86      0.86       876
weighted avg       0.88      0.87      0.87       876



In [10]:
print(confusion_matrix)

[[ 85   3   0   2   0   1   2]
 [  0  91   1   3   3   0   9]
 [  2   0  78   3   3   1   2]
 [  3   2   2 116  10   1   5]
 [  4   4   2   8 102   1   2]
 [  1   3   0   1   2  34   4]
 [  6   1   1   9   3   0 260]]
